In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import gk
import importlib
importlib.reload(gk)

In [ ]:
path = "run-01"
sim = "rt_vlasov_sr_twostream_1x1v"
steps = np.arange(1)

n0 = 1.0
vb = 1.0 / 20.0
gammab = 1.0 / np.sqrt(1.0 - vb**2)
ub = gammab * vb
deltaub = ub / 25.0
omegaPe = np.sqrt(n0)

# fields = gk.ReadFields(path, sim, step)
df = gk.ReadDist(path, sim, "elc", steps[-1])

fig = plt.figure(figsize=(12, 4), dpi=300)
gs = fig.add_gridspec(1, 2)
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])
ax1.pcolormesh(
    df["x"] * omegaPe / ub,
    df["ux"] / ub,
    # np.log10(df["elc"][:, :] / df["elc"][:, :].max()),
    # vmin=-6,
    # vmax=0,
    df["elc"][:, :] / df["elc"][:, :].max(),
    vmin=0,
    vmax=1,
    cmap="Greys",
)
ax1.set(
    ylim=(-4, 4),
    xlabel=r"$x \omega_{pe}/u_b$",
    ylabel=r"$u_x/u_b$",
)

for step in steps:
    df = gk.ReadDist(path, sim, "elc", step)
    print(np.sum(df["elc"]))
    ux_mean = (df["ux"][1:] + df["ux"][:-1]) * 0.5
    df_du = np.mean(df["elc"], axis=1)
    mask = ux_mean >= 0
    df_du = df_du[mask] + df_du[~mask][::-1]
    ux_mean = ux_mean[mask]
    ax2.plot(
        (np.sqrt(1 + ux_mean**2) - 1) / (gammab - 1),
        np.sqrt(1 + ux_mean**2) * df_du,
        c=plt.cm.coolwarm(step / steps[-1]),
    )
    # df_dlogv2 = Ux_mean * (1 + Ux_mean**2) * df_du
    # Vx_mean = Ux_mean / np.sqrt(1 + Ux_mean**2)
    # ax2.plot((Vx_mean / vb) ** 2, (Vx_mean / vb) ** 2 * df_dlogv2, c=plt.cm.coolwarm(step / steps[-1]))
ax2.set(
    xscale="log",
    yscale="log",
    xlim=(0.1, 20),
    ylim=(1e-4, 100),
    xlabel=r"$(\gamma - 1)/(\gamma_b - 1)$",
    ylabel=r"$\gamma dn/du$",
)